# Centrality measures experiments

Author: [picorana](https://picorana.github.io/)

Github repository: [link](https://github.com/picorana/Graph_Centrality_Measures)

If you are looking at this via github, please use this link to visualize it in the proper way: [this notebook on nbviewer](https://nbviewer.jupyter.org/github/picorana/Graph_Centrality_Measures/blob/master/notebook.ipynb)

## Centrality

Centrality measures address the question of defining the concept of importance of a node in a graph.  
We can answer to the question of "what node is the most important in a graph?" in many different ways, depending on the context. 

The question is infact perfectly comparable to asking "Which one of my friends is the most famous one in my circle of friends?", "Which research paper is the most relevant?", or "Which github repository is better?"

For this purpose, we can choose among different ways of measuring it. 
Possible types of measures are:
* Degree centrality
* Closeness centrality
* Betweeness centrality
* Eigenvector centrality
* PageRank centrality

The aim of this notebook is to explain a possible implementation for computing eigenvector centrality, and showing a comparison with degree centrality.

### Initial setup

We are going to use the networkx library for storing and managing the graph, plus scipy and numpy for working with matrices.

In [1]:
import networkx as nx
import scipy as sc
import numpy as np

For the sake of visualization, we are going to use plotly. Since talking about visualization is not the purpose of this notebook, I made a module to manage all the visualization-related functions and to store plotly configurations. The code of this module can still be found in the same repository of this notebook, if you are interested in taking a look.

In [2]:
#import plotly_config as pc
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import *

init_notebook_mode(connected=True)

In [3]:
def loadLayout():
    
    l = Layout(
        title='<br>Network graph made with Python',
        titlefont=dict(size=16),
        showlegend=False,
        hovermode='closest',
        margin=dict(b=20,l=5,r=5,t=40),
        annotations=[ dict(
            text="Python code: <a href='https://plot.ly/ipython-notebooks/network-graphs/'> https://plot.ly/ipython-notebooks/network-graphs/</a>",
            showarrow=False,
            xref="paper", yref="paper",
            x=0.005, y=-0.002 ) ],
        xaxis=XAxis(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=YAxis(showgrid=False, zeroline=False, showticklabels=False))
    
    return l


edge_trace = Scatter(
    x=[],
    y=[],
    line=Line(width=0.5,color='#888'),
    hoverinfo='none',
    mode='lines')

node_trace = Scatter(
    x=[],
    y=[],
    text=[],
    mode='markers',
    hoverinfo='text',
    marker= Marker(
        showscale=True,
        colorscale='Portland',
        reversescale=True,
        color=[],
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line=dict(width=2)))

def drawSpringLayoutGraph(test_G):
    
    pos = nx.spring_layout(test_G)
    
    for edge in test_G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_trace['x'] += [x0, x1, None]
        edge_trace['y'] += [y0, y1, None]

    for node in test_G.nodes():
        x, y = pos[node]
        node_trace['x'].append(x)
        node_trace['y'].append(y)
        
    node_trace['marker']['size'] = []
    
    for node in test_G.nodes():
        node_trace['marker']['size'].append(20)

    for node, adjacencies in enumerate(test_G.adjacency_list()):
        node_trace['marker']['color'].append(len(adjacencies))
        node_info = '# AAA: '+str(len(adjacencies)) + '\n node number: ' + str(node)
        node_trace['text'].append(node_info)

    fig = Figure(data = Data([edge_trace, node_trace]), layout = loadLayout())

    iplot(fig)

# Degree Centrality

The degree of a node is just the number of connections it has with other nodes. In the context of a social network, if an user is a node, its degree could be interpreted as the number of friends he has.

Degree Centrality is a simple form of interpreting how relevant a node is.

Let us play with a toy graph in the beginning, here we define it:

In [4]:
test_G = nx.Graph()
test_G.add_edge(0, 1)
test_G.add_edge(1, 2)
test_G.add_edge(2, 3)
test_G.add_edge(2, 10)
test_G.add_edge(3, 0)
test_G.add_edge(3, 1)
test_G.add_edge(1, 4)
test_G.add_edge(4, 5)
test_G.add_edge(5, 6)
test_G.add_edge(5, 7)
test_G.add_edge(5, 8)
test_G.add_edge(5, 9)
test_G.add_edge(5, 11)

And here we draw it:

In [5]:
#drawSpringLayoutGraph(test_G)

## Eigenvector Centrality

Eigenvector Centrality is just another way of computing the influence of a node in a network. The main idea behind this is that having connections to many nodes with high centrality values is more important than just having many connections.

If we see it in the context of a social network, an interpretation of this may be that a person with famous friends is more famous than a person with many friends.

### Computing Eigenvector Centrality
Let $A = (a_{v,t})$ be the adjacency matrix of a graph $G$.  
Let $M(v)$ be the set of neighbors of $v$.  
Let $\lambda$ be the greatest eigenvalue of $A$.  

The eigenvector centrality score of a vertex $v \in G$ can be defined as:

$$ x_v = \frac{1}{\lambda}\sum\limits_{t \in M(v)} x_t = \frac{1}{\lambda}\sum\limits_{t \in G} a_{v,t}x_t$$

First, we compute the adjacency matrix. Since our graph is undirected, our adjacency matrix will be symmetric.

In [6]:
def create_adjacency_matrix(test_G):
    nodelist = test_G.nodes()
    nlen = len(nodelist)

    # define an index of the nodes in the graph
    index = dict(zip(nodelist,range(nlen)))

    # initialize a matrix full of zeros
    M = np.zeros((nlen,nlen), dtype=int)

    # for each edge in G
    for u,v in test_G.edges_iter():
        i,j = index[u], index[v]

        # write 1 in each cell corresponding to adjacent nodes
        M[i,j] = 1

        # if the graph is undirected, this matrix is going to be symmetric
        if not test_G.is_directed():
            M[j,i] = M[i,j]
            
    return M

M = create_adjacency_matrix(test_G)
print M

[[0 1 0 1 0 0 0 0 0 0 0 0]
 [1 0 1 1 1 0 0 0 0 0 0 0]
 [0 1 0 1 0 0 0 0 0 0 1 0]
 [1 1 1 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 1 0 1 1 1 1 0 1]
 [0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0]]


As you can see, this matrix represents the connections between nodes.

Now, this matrix is very wasteful: it contains a lot of information that we don't really need with all those zero cells. We are, infact, just interested in the adjacent nodes, so we could as well use a much more efficient sparse matrix. Scipy conveniently offers us a function to directly transform a non-sparse matrix to a sparse one, so we are going to use that.

In [7]:
M_sparse = sc.sparse.coo_matrix(M)

print M_sparse

  (0, 1)	1
  (0, 3)	1
  (1, 0)	1
  (1, 2)	1
  (1, 3)	1
  (1, 4)	1
  (2, 1)	1
  (2, 3)	1
  (2, 10)	1
  (3, 0)	1
  (3, 1)	1
  (3, 2)	1
  (4, 1)	1
  (4, 5)	1
  (5, 4)	1
  (5, 6)	1
  (5, 7)	1
  (5, 8)	1
  (5, 9)	1
  (5, 11)	1
  (6, 5)	1
  (7, 5)	1
  (8, 5)	1
  (9, 5)	1
  (10, 2)	1
  (11, 5)	1


Now we need to compute the eigenvectors and the resulting eigenvector centrality.

In [8]:
def compute_eigen_centr(M_sparse):
    eigenvalues, eigenvectors = sc.sparse.linalg.eigs(M_sparse.asfptype())
    index_of_greatest_eigenvalue = eigenvalues.real.argmax(axis=0)  
    dominant_eigenvector = eigenvectors[:,index_of_greatest_eigenvalue]

    eigenvector_centrality = {}

    for i in range (0, test_G.number_of_nodes()):
        eigenvector_centrality[i] = abs(dominant_eigenvector[i].real)

    factor = 1.0 / sum(eigenvector_centrality.values())
    for k in eigenvector_centrality: 
        eigenvector_centrality[k] = eigenvector_centrality[k] * factor
        
    return eigenvector_centrality

print compute_eigen_centr(M_sparse)

{0: 0.11944518672642711, 1: 0.18090047353384806, 2: 0.13670202819104554, 3: 0.15528231721527438, 4: 0.097721382250572433, 5: 0.094139885154717648, 6: 0.033447744709516961, 7: 0.033447744709517162, 8: 0.033447744709517058, 9: 0.033447744709517176, 10: 0.048570003380529608, 11: 0.03344774470951694}


Now, the one below is just one way of doing it, that assumes that the matrix is small enough for us to actually compute the eigenvectors. 

In other circumstances, we would have to use an approximation. The **power method**, shown in the code below, iterates through different cycles with the purpose of finding the dominant eigenvector and eigenvalue.

The method starts by using a random vector $b_0$.  
It assumes that:
* A has an eigenvalue that is strictly greater in magnitude than its other eigenvalues
* The starting vector $b_0$ has a nonzero component in the direction of an eigenvector associated with the dominant eigenvalue.

At every iteration, $b_0$ is multiplied by the adjacence matrix $A$ and normalized:

$$b_{k+1} = \frac{Ab_k}{||Ab_k||}$$

$b_k$ should converge to an eigenvector associated to the dominant eigenvalue.

In [9]:
def power_iteration(A):
    # start with a random vector
    bk = np.random.rand(A.shape[0])
    
    num_simulations = 100

    for _ in range(num_simulations):
        bk = np.dot(A, bk) / np.linalg.norm(np.dot(A, bk))

    return bk

def normalize_vector(vec):
    return vec / np.linalg.norm(vec, ord=1)
        
print normalize_vector(power_iteration(M))

[ 0.1194452   0.1809007   0.13670204  0.15528239  0.09772123  0.09414036
  0.03344761  0.03344761  0.03344761  0.03344761  0.04857005  0.03344761]


In the end, we finally visually represent the values to get a better idea of how they compare with what we have seen previously.

In [10]:
#drawSpringLayoutGraphFromDict(test_G, eigenvector_centrality)

In the graph above, you can actually compare the difference between degree cenrality and eigenvector centrality because of how they are represented: the size of a node represents its eigenvector centrality, while the color represents its degree centrality.

It is interesting to notice that node 5 has 6 edges towards other nodes, and has therefore the highest value in degree centrality, but doesn't keep the same relevance if instead we consider eigenvector centrality, for which node 1 detains the highest value, because it has more connections with more connected nodes. 

# Reddit test

Now, let's load our actual data.

In [11]:
G = nx.read_edgelist(open('data/edgelist_weighted_clean.txt', 'rb'))

In [12]:
eigenvector_centrality = nx.eigenvector_centrality(G)

In [13]:
print G.number_of_nodes()
print G.number_of_edges()

6909
518493


In [14]:
def trim_graph_edges_for_visualization(G):
    
	for node in G.nodes():
		for edge in G.edges(node):
			if G[edge[0]][edge[1]]['weight'] < 0.05:
				G.remove_edge(edge[0], edge[1])

	solitary= [n for n,d in G.degree_iter() if d==0]
 	G.remove_nodes_from(solitary)
	return G

def drawSpringLayoutGraphFromDict(G2, node_sizes):
    
    #edge_trace, node_trace = init_edge_node_trace()
    
    pos = nx.spring_layout(G2)
    
    for edge in G2.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_trace['x'] += [x0, x1, None]
        edge_trace['y'] += [y0, y1, None]

    for node in G2.nodes():
        x, y = pos[node]
        node_trace['x'].append(x)
        node_trace['y'].append(y)
        
    node_trace['marker']['size'] = []
    
    for node in G2.nodes():
        if node_sizes[node]==0: print node
        node_trace['marker']['size'].append(node_sizes[node]*500)
        node_info = 'name: ' + str(node)
        node_trace['text'].append(node_info)

    for node, adjacencies in enumerate(G2.adjacency_list()):
        node_trace['marker']['color'].append(len(adjacencies))
        

    fig = Figure(data = Data([edge_trace, node_trace]), layout = loadLayout())

    iplot(fig)

In [15]:
G2 = trim_graph_edges_for_visualization(G)

In [16]:
print G2.number_of_nodes()
print G2.number_of_edges()

1721
4124


In [17]:
drawSpringLayoutGraphFromDict(G2, eigenvector_centrality)